# Import bibs

In [18]:
from utilities.Functionnalities import *

## download node modules to use for collecting data with nodejs
### npm install ccxt
### npm install csv-writer

In [19]:
install_package_if_needed("ccxt")

ccxt is already installed.


In [20]:
install_package_if_needed("csv-writer")

csv-writer is already installed.


## collecting data with nodejs
### node ./database/dl_for_quick_analysis.js

In [21]:
command = 'node ./database/dl_for_quick_analysis.js'
execute_terminal_command(command)

database/quick_analysis is deleted!

Loading 1/590 coins downloaded
Loading 2/590 coins downloaded
Loading 3/590 coins downloaded
Loading 4/590 coins downloaded
Loading 5/590 coins downloaded
Loading 6/590 coins downloaded
Loading 7/590 coins downloaded
Loading 8/590 coins downloaded
Loading 9/590 coins downloaded
Loading 10/590 coins downloaded
Loading 11/590 coins downloaded
Loading 12/590 coins downloaded
Loading 13/590 coins downloaded
Loading 14/590 coins downloaded
Loading 15/590 coins downloaded
Loading 16/590 coins downloaded
Loading 17/590 coins downloaded
Loading 18/590 coins downloaded
Loading 19/590 coins downloaded
Loading 20/590 coins downloaded
Loading 21/590 coins downloaded
Loading 22/590 coins downloaded
Loading 23/590 coins downloaded
Loading 24/590 coins downloaded
Loading 25/590 coins downloaded
Loading 26/590 coins downloaded
Loading 27/590 coins downloaded
Loading 28/590 coins downloaded
Loading 29/590 coins downloaded
Loading 30/590 coins downloaded
Loading 31/5

## remove unecessary files
### just keep the csv files

In [22]:
directory_path = "./database/quick_analysis"
remove_non_csv_files(directory_path)

# Process Started

In [23]:
mypath = "./database/quick_analysis"
file_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]
df_list = {}
for file in file_list:
    df_list[file[:-9]] = get_historical_from_path(mypath+"/"+file)

print("All data loaded")

All data loaded


Variables

In [24]:
# entries
dateObservation = 24
nbPool = 6
delta_hour = "8h"
n_i = "N"
star_time = "2023-01-01"
star_hour = "10:00:00"
end_time = "2023-03-03"
end_hour = "15:00:00"

In [25]:
sttDate = f"{star_time} {star_hour}"
ennDate = f"{end_time} {end_hour}"
client = Client()
crypto = {}

In [26]:
df_metric = get_analyisis_from_window(df_list,dateObservation).sort_values(by="volume_evolution", ascending=False)
dfVe = df_metric.iloc[:nbPool]
market = list(dfVe.index)

/home/keranis/work&study/Stage/utilities/Functionnalities.py:273: RuntimeWarning: invalid value encountered in scalar divide
  return_vs_vol = perf / volatility
/home/keranis/work&study/Stage/utilities/Functionnalities.py:275: RuntimeWarning: invalid value encountered in scalar divide
  volume_evolution = last_volume / mean_volume
/home/keranis/work&study/Stage/utilities/Functionnalities.py:278: RuntimeWarning: invalid value encountered in scalar divide
  volatility_evolution = last_volatility / volatility
/home/keranis/work&study/Stage/utilities/Functionnalities.py:279: RuntimeWarning: invalid value encountered in scalar divide
  last_return_vs_vol = last_perf / last_volatility
/home/keranis/work&study/Stage/utilities/Functionnalities.py:273: RuntimeWarning: invalid value encountered in scalar divide
  return_vs_vol = perf / volatility
/home/keranis/work&study/Stage/utilities/Functionnalities.py:275: RuntimeWarning: invalid value encountered in scalar divide
  volume_evolution = last_

In [27]:
x = ""
for elm in market :
    try :
        #get the values from binance as they are and convert them to a dataframe we can work with
        if elm.endswith("-USDT"):
            x = elm[:-5]
        else:
            x = elm
        x = x.lower()+'/usdt'
        x = ''.join(char for char in x if not char.isdigit())
        crypto[x] = client.get_historical_klines(x.replace("/", "").upper(), delta_hour, sttDate, ennDate)
        crypto[x] = pd.DataFrame(data=crypto[x],
                                 columns=['timestamp', x[:-5] + '_open', 'high', 'low', x[:-5] + '_close', 'volume','close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])
        crypto[x] = crypto[x][['timestamp', x[:-5] + '_open', x[:-5] + '_close']]
        crypto[x] = convert_time(crypto[x])
        crypto[x] = crypto[x].astype({x[:-5] + '_open': 'float64',x[:-5] + '_close': 'float64'})
        crypto[x] = crypto[x].set_index('timestamp')
    except :
        print(f"{x} :  Please Try another name of this crypto !")
array_mauvais_shape = detection_mauvais_shape(crypto)
crypto = correction_shape(crypto, array_mauvais_shape)
for elm in array_mauvais_shape:
    crypto[elm]['timestamp'] = generation_date(crypto[elm], int(delta_hour[:1]))
    crypto[elm] = crypto[elm].set_index('timestamp')

In [28]:
crypto = variationN(crypto, n_i)
crypto

{'epx/usdt':                            epx_open  epx_close  Variation_epx  Variation_N_epx
 timestamp                                                                     
 2023-01-01 17:00:00+00:00  0.000291   0.000291       1.000000         1.000000
 2023-01-02 01:00:00+00:00  0.000291   0.000294       1.010309         1.010309
 2023-01-02 09:00:00+00:00  0.000295   0.000294       0.996610         0.996610
 2023-01-02 17:00:00+00:00  0.000294   0.000296       1.006803         1.006803
 2023-01-03 01:00:00+00:00  0.000297   0.000295       0.993266         0.993266
 ...                             ...        ...            ...              ...
 2023-03-02 01:00:00+00:00  0.000356   0.000351       0.985955         0.985955
 2023-03-02 09:00:00+00:00  0.000351   0.000345       0.982906         0.982906
 2023-03-02 17:00:00+00:00  0.000344   0.000347       1.008721         1.008721
 2023-03-03 01:00:00+00:00  0.000346   0.000331       0.956647         0.956647
 2023-03-03 09:00:00+00:00  

In [59]:
cryptos = crypto
for cr in cryptos :
    for i in range(len(cryptos[cr].index)):
        if(i == 0):
            try :
                y=cryptos[cr][cr[:-5] + "_close"][0]
                print(y)
            except:
                print("no idea")

0.000291
0.00842
0.00015051
0.003
no idea
0.3339


In [29]:
crypto = coeffMulti(crypto)
crypto = mergeCryptoTogether(crypto)
crypto, maxis = botMax(crypto)
crypto = botMaxVariation2(crypto, maxis)
crypto = coeffMultiBotMax(crypto)
coefMulti = coefmultiFinal(crypto)

KeyError: 0

In [ ]:
plot_courbes2(coefMulti)

In [ ]:
coefMulti.tail(1)